In [1]:
import pandas as pd
import os
from json import loads

In [2]:
def load_json(filename):
    with open(filename) as f:
        _temp_dict = []
        for line in f:
            _temp_dict.append(loads(line))
    return pd.DataFrame.from_dict(_temp_dict, orient='columns')
        

df = pd.DataFrame()
for _file in os.listdir():
    if not _file.startswith('modal_'):
        continue
    
    df = pd.concat([df, load_json(_file)])

In [3]:
od_matrix = load_json('./od_matrix_per_zone.json')

In [8]:
od_matrix.head()

,dayofweek,origin_zone,destination_zone,period,trip_count
0,5,2,2,afternoon_peak,418
1,5,2,2,afternoon_valley,751
2,5,2,2,morning_valley,496
3,5,2,2,night,326
4,5,2,3,afternoon_peak,197


In [5]:
df.shape

(1253025, 7)

In [6]:
df.dropna().shape

(1162098, 7)

In [54]:
(df
 .merge(od_matrix, how='right')
 .dropna()
 .to_json('od_matrix_model_merged.json', orient='records')
)

In [10]:
# 90% of coverage by dayofweek, period
(df
 .merge(od_matrix, how='right')
 .sort_values('trip_count', ascending=False)
 .groupby(['dayofweek', 'period'])
 .apply(lambda x: x.query('trip_count.cumsum() < trip_count.sum()*0.9'))
 .to_json('od_matrix_model_merged_90.json', orient='records')
)

In [11]:
# 75% of coverage
(df
 .merge(od_matrix, how='right')
 .sort_values('trip_count', ascending=False)
 .groupby(['dayofweek', 'period'])
 .apply(lambda x: x.query('trip_count.cumsum() < trip_count.sum()*0.75'))
 .to_json('od_matrix_model_merged_75.json', orient='records')
)